In [1]:
import glob
import pandas as pd
# Backtesting Libs
from backtesting import set_bokeh_output
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

C:\Users\RudraRV\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
set_bokeh_output(notebook=False)

In [3]:
class CheckSignal(Strategy):
    def init(self):
        pass

    def next(self):
        self.signal = (self.data.signal)
        if not self.position:
            if self.signal[-1] == 1:
#                 print('buy')
                self.buy()
#                 self.buy(limit = self.data.Close)

            elif self.signal[-1] == -1:
#                 print('sell')
                self.sell()
#                 self.sell(limit = self.data.Close)
        
        else:
            if self.signal[-1] != 0:
#                 print('close')
                self.position.close()

In [4]:
def backtest_plot(df,strategy):
    initial_cash = 100_000
    ldf = len(df)
    idf = 0
    while idf < ldf:
        ulim = min(ldf-idf,10_000)
        bt = Backtest(df.iloc[idf:idf+ulim], strategy, cash=100_000, commission=0.1*2/100, # commission on 2 transactions B&S
                  exclusive_orders=False)
        bt.run()
        bt.plot()
        idf += 10_000

In [5]:
def calculate_sma(df, window, column_name='close'):
    sma = df[column_name].rolling(window=window, min_periods=1).mean()
    return sma
def calculate_ema(df, window, column_name='close'):
    ema = df[column_name].ewm(span=window, adjust=False).mean()
    return ema

In [6]:
datasets_path_lst = (glob.glob(r".\InterIIT Competition Data\*")) 
datasets_path_dct = dict()
for dataset_path in datasets_path_lst:
    datasets_path_dct[dataset_path[36:-4]] = dataset_path
datasets_path_dct

{'5m': '.\\InterIIT Competition Data\\btcusdt_5m.csv',
 '3m': '.\\InterIIT Competition Data\\btcusdt_3m.csv',
 '30m': '.\\InterIIT Competition Data\\btcusdt_30m.csv',
 '1h': '.\\InterIIT Competition Data\\btcusdt_1h.csv',
 '15m': '.\\InterIIT Competition Data\\btcusdt_15m.csv'}

In [7]:
df = pd.read_csv(datasets_path_dct['1h'])

In [8]:
df['SMA'] = calculate_sma(df, 15)
df['EMA'] = calculate_ema(df, 15)

In [9]:
df

,datetime,open,high,low,close,volume,SMA,EMA
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13529.010000,13529.010000
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13366.035000,13488.266250
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13354.083333,13468.505469
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13368.070000,13461.196035
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13414.658000,13478.672781
...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42160.364000,42284.919955
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42197.310667,42331.704961
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42230.782667,42379.383090
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42279.950000,42423.121454


In [10]:
df['signal'] = 0
df

,datetime,open,high,low,close,volume,SMA,EMA,signal
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13529.010000,13529.010000,0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13366.035000,13488.266250,0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13354.083333,13468.505469,0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13368.070000,13461.196035,0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13414.658000,13478.672781,0
...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42160.364000,42284.919955,0
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42197.310667,42331.704961,0
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42230.782667,42379.383090,0
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42279.950000,42423.121454,0


In [11]:
buy_condition = (df['SMA'] > df['EMA']) & (df['SMA'].shift(-1) < df['EMA'].shift(-1))
sell_condition = (df['SMA'] < df['EMA']) & (df['SMA'].shift(-1) > df['EMA'].shift(-1))
df.loc[buy_condition, 'signal'] = 1
df.loc[sell_condition, 'signal'] = -1

In [12]:
df.drop(columns = ['SMA','EMA'], inplace = True)
df

,datetime,open,high,low,close,volume,signal
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,0
...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,0
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,0
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,0
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,0


In [13]:
df.columns = ['timestamp','Open','High','Low','Close','Volume','signal']
df = df.loc[:,['timestamp','signal','Open','High','Low','Close','Volume']]
df1 = df.copy()
df.index = pd.DatetimeIndex(df['timestamp'])
df.drop(columns = ['timestamp'], inplace = True)
df

,signal,Open,High,Low,Close,Volume
timestamp,,,,,,
2018-01-01 05:30:00,0,13715.65,13715.65,13400.01,13529.01,443.356199
2018-01-01 06:30:00,0,13528.99,13595.89,13155.38,13203.06,383.697006
2018-01-01 07:30:00,0,13203.00,13418.43,13200.00,13330.18,429.064572
2018-01-01 08:30:00,0,13330.26,13611.27,13290.00,13410.03,420.087030
2018-01-01 09:30:00,0,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...,...
2022-01-12 01:30:00,0,42972.04,43095.26,42692.19,42800.38,1219.601780
2022-01-12 02:30:00,0,42797.62,42823.69,42643.74,42659.20,702.103800
2022-01-12 03:30:00,0,42664.71,42776.14,42597.41,42713.13,561.859930


In [14]:
bt = Backtest(df.iloc[:], CheckSignal, cash=100_000, commission=0.1*2/100, # commission on 2 transactions B&S
              exclusive_orders=False)
stats = bt.run()

In [15]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7,12,21,13194.11888,13399.23,-1.435778e+03,-0.015546,2018-01-01 17:30:00,2018-01-02 02:30:00,0 days 09:00:00
1,-7,31,37,13229.55786,13489.04,-1.816375e+03,-0.019614,2018-01-02 12:30:00,2018-01-02 18:30:00,0 days 06:00:00
2,-6,54,70,14950.92822,14926.96,1.438093e+02,0.001603,2018-01-03 11:30:00,2018-01-04 03:30:00,0 days 16:00:00
3,-6,76,89,15089.73006,14648.00,2.650380e+03,0.029274,2018-01-04 10:30:00,2018-01-04 23:30:00,0 days 13:00:00
4,-6,102,104,14922.43532,15429.60,-3.042988e+03,-0.033987,2018-01-05 12:30:00,2018-01-05 14:30:00,0 days 02:00:00
...,...,...,...,...,...,...,...,...,...,...
1497,-5971,35109,35111,41791.25000,41566.48,1.342102e+06,0.005378,2022-01-08 03:30:00,2022-01-08 05:30:00,0 days 02:00:00
1498,-5999,35125,35135,41821.18002,41679.74,8.484987e+05,0.003382,2022-01-08 19:30:00,2022-01-09 05:30:00,0 days 10:00:00
1499,-6076,35145,35153,41426.98000,42094.48,-4.055730e+06,-0.016113,2022-01-09 15:30:00,2022-01-09 23:30:00,0 days 08:00:00
1500,-5957,35160,35179,41571.86964,41297.15,1.636505e+06,0.006608,2022-01-10 06:30:00,2022-01-11 01:30:00,0 days 19:00:00


In [16]:
stats

Start                     2018-01-01 05:30:00
End                       2022-01-12 05:30:00
Duration                   1472 days 00:00:00
Exposure Time [%]                   54.300159
Equity Final [$]              245462798.28494
Equity Peak [$]               257737271.47046
Return [%]                      245362.798285
Buy & Hold Return [%]               215.43328
Return (Ann.) [%]                  591.150124
Volatility (Ann.) [%]              377.214701
Sharpe Ratio                         1.567145
Sortino Ratio                       24.595325
Calmar Ratio                        18.502706
Max. Drawdown [%]                  -31.949388
Avg. Drawdown [%]                   -2.005425
Max. Drawdown Duration      111 days 16:00:00
Avg. Drawdown Duration        2 days 11:00:00
# Trades                                 1502
Win Rate [%]                        49.267643
Best Trade [%]                      24.722833
Worst Trade [%]                    -12.149123
Avg. Trade [%]                    

In [17]:
# Uncomment to generate plots
backtest_plot(df,CheckSignal)

C:\Users\RudraRV\AppData\Local\Temp\ipykernel_23048\496147577.py:10: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
C:\Users\RudraRV\AppData\Local\Temp\ipykernel_23048\496147577.py:10: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
C:\Users\RudraRV\AppData\Local\Temp\ipykernel_23048\496147577.py:10: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
C:\Users\RudraRV\AppData\Local\Temp\ipykernel_23048\496147577.py:10: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


In [18]:
df1.columns = ['timestamp','signal','open','high','low','close','volume']
submission = df1
submission 

,timestamp,signal,open,high,low,close,volume
0,2018-01-01 05:30:00,0,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 06:30:00,0,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 07:30:00,0,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 08:30:00,0,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 09:30:00,0,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,0,42972.04,43095.26,42692.19,42800.38,1219.601780
35204,2022-01-12 02:30:00,0,42797.62,42823.69,42643.74,42659.20,702.103800
35205,2022-01-12 03:30:00,0,42664.71,42776.14,42597.41,42713.13,561.859930
35206,2022-01-12 04:30:00,0,42713.12,42886.28,42633.97,42729.29,681.142010


In [19]:
submission.describe()

,signal,open,high,low,close,volume
count,35208.000000,35208.000000,35208.000000,35208.000000,35208.000000,35208.000000
mean,0.000000,18539.414637,18649.115725,18421.316673,18540.235786,2280.268987
std,0.292103,17824.522767,17932.147494,17709.969405,17824.970956,2188.271567
min,-1.000000,3172.620000,3184.750000,3156.260000,3172.050000,0.000000
25%,0.000000,7101.045000,7140.000000,7062.735000,7101.727500,1066.353192
50%,0.000000,9471.150000,9512.000000,9423.960000,9470.990000,1663.188535
75%,0.000000,31580.232500,31800.450000,31275.312500,31580.440000,2710.796384
max,1.000000,68635.120000,69000.000000,68451.190000,68633.690000,47255.762685
